In [ ]:
pip uninstall tensorflow

Uninstalling tensorflow-2.4.1:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.7/dist-packages/tensorflow-2.4.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.4.1


# Mục mới

In [ ]:
pip install tensorflow-gpu==2.2.0

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Convolution2D, MaxPooling2D, Flatten, Dense,Activation,BatchNormalization,Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from  tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
print(tf.__version__)
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import json

2.4.1


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ls "/content/gdrive/My Drive/data"

train_generator = ImageDataGenerator(
    #rescale = 8/10,
    #fill_mode = 'nearest',
    fill_mode = "constant",
    #fill_mode = "reflect",
    #samplewise_center=True,
    #samplewise_std_normalization=True,
    #validation_split=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    vertical_flip=True,
    rotation_range=25,
)

batch_size = 8

train_gen = train_generator.flow_from_directory(
    "/content/gdrive/My Drive/data",
    #color_mode="grayscale",
    target_size = (224,224),
    batch_size = batch_size, 
    #class_mode='input',
    subset='training',
    shuffle=True
)


val_gen = train_generator.flow_from_directory("/content/gdrive/My Drive/data",
    #color_mode="grayscale",
    target_size = (224,224),
    batch_size = batch_size, 
    #class_mode='input',
    subset='validation',
    shuffle=True
)


dict_ = val_gen.class_indices
class_dict = dict()
    
for key in dict_:
  class_dict[dict_[key]] = key

with open("class.json","w") as f:
  json.dump(class_dict, f)
check_point = ModelCheckpoint("/content/gdrive/My Drive/face_detector.h5"
                                ,monitor="loss"
                                ,mode="min"
                                ,save_best_only = True,
                                verbose=1)


17103100361  17103100383  17103100397  17103100413  17103100428  17103100460
17103100364  17103100385  17103100398  17103100414  17103100432
17103100365  17103100389  17103100401  17103100418  17103100434
17103100368  17103100390  17103100404  17103100422  17103100444
17103100373  17103100394  17103100409  17103100427  17103100447
Found 345 images belonging to 26 classes.
Found 0 images belonging to 26 classes.


In [5]:


num_class = train_gen.num_classes
initial_model = tf.keras.applications.VGG16( weights='imagenet',
    include_top=False, input_tensor=None,
    input_shape=(224,224,3), pooling=None   
)


for layer in initial_model.layers:
    layer.trainable = False
last = initial_model.output

last = Flatten()(last)

last = Dense(4096, activation='relu')(last)
last = Dropout(0.25)(last)
last = Dense(4096, activation='relu')(last)
last = Dropout(0.25)(last)
last = BatchNormalization()(last)
#last = Dense(256, activation='relu')(last)
#last = Dense(158, activation='relu')(last)
last = Dense(num_class, activation='softmax')(last)

model = Model(inputs = initial_model.input, outputs =last )

print(model.summary())
#current_dir = os.path.dirname(os.path.realpath(__file__))


"""
val_generator = ImageDataGenerator(
    #rescale = 1./255,
    #fill_mode = 'nearest',
    fill_mode = "constant",
    validation_split=0.3,
)
"""
if(os.path.isfile("/content/gdrive/My Drive/face_detector_uneti.h5")):
  model = load_model("/content/gdrive/My Drive/face_detector_uneti.h5")


earlystop = EarlyStopping(
monitor='loss',
min_delta=0.4,
patience=10, 
verbose=1,
restore_best_weights=True
)

callbacks = [earlystop, check_point]
#callbacks = [check_point]
model.compile( loss = 'categorical_crossentropy',
              optimizer = "nadam",
              metrics = ['accuracy']
)
print(len(train_gen))
hist = model.fit(
    train_gen,
    #validation_data = val_gen,
    epochs = 150,
    callbacks = callbacks,
    
)

model.save("/content/gdrive/My Drive/face_detector_uneti.h5")

"""
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy"])
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Loss","Validation Loss"])
plt.show()
"""

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

'\nplt.plot(hist.history["accuracy"])\nplt.plot(hist.history[\'val_accuracy\'])\nplt.title("model accuracy")\nplt.ylabel("Accuracy")\nplt.xlabel("Epoch")\nplt.legend(["Accuracy","Validation Accuracy"])\nplt.show()\n\nplt.plot(hist.history[\'loss\'])\nplt.plot(hist.history[\'val_loss\'])\nplt.title("model loss")\nplt.ylabel("Loss")\nplt.xlabel("Epoch")\nplt.legend(["Loss","Validation Loss"])\nplt.show()\n'